In [1]:
# notebook import

from ipynb.fs.full.data_processing import *
from ipynb.fs.full.mi_calculation import shannon_entropy_fn
from ipynb.fs.full.mi_calculation import calculate_mi

well_fpath_list: <class 'list'> 
 ['/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_01.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_03.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_04.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_05.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_06.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_07.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_09.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_10.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_11.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_12.csv', '/Users/sarahfisher/Documents/proje

feature_df: <class 'pandas.core.frame.DataFrame'> 
                 gfp         rfp  area
0        528.690625  237.327083   960
1        403.250916  216.184982   546
2        384.769737  210.777961  1216
3        346.063130  202.326790  1885
4        330.310096  198.775240   832
...             ...         ...   ...
223548   901.914583  262.665972  1440
223549  1032.986538  276.790385   520
223550   921.374693  263.346437   814
223551   938.197815  267.918919  1739
223552   981.538084  263.349713  1221

[2720208 rows x 3 columns]
ncells_list_dict: <class 'dict'> 
 {1: [56, 52, 63, 39, 42, 72, 112, 121, 147, 118, 28, 39, 35], 2: [41, 72, 46, 42, 54, 83, 98, 133, 130, 105, 33, 80, 57], 3: [56, 63, 38, 36, 42, 64, 116, 116, 138, 98, 63, 72, 63], 4: [32, 73, 35, 33, 37, 66, 99, 107, 127, 105, 41, 73, 48], 5: [66, 50, 27, 46, 52, 78, 101, 114, 131, 98, 52, 88, 61], 6: [42, 62, 46, 35, 31, 82, 114, 113, 108, 120, 41, 77, 69], 7: [47, 82, 34, 40, 39, 71, 86, 114, 143, 89, 45, 68, 67], 8: [59,

bin_dict: <class 'dict'> 
 {'gfp': array([ 226.76648352,  311.67626067,  396.58603782,  481.49581497,
        566.40559213,  651.31536928,  736.22514643,  821.13492358,
        906.04470074,  990.95447789, 1075.86425504, 1160.77403219,
       1245.68380935, 1330.5935865 , 1415.50336365, 1500.4131408 ,
       1585.32291796, 1670.23269511, 1755.14247226, 1840.05224941,
       1924.96202657, 2009.87180372, 2094.78158087, 2179.69135802]), 'rfp': array([ 169.59401709,  223.45627941,  277.31854173,  331.18080405,
        385.04306637,  438.90532869,  492.76759101,  546.62985332,
        600.49211564,  654.35437796,  708.21664028,  762.0789026 ,
        815.94116492,  869.80342723,  923.66568955,  977.52795187,
       1031.39021419, 1085.25247651, 1139.11473883, 1192.97700115,
       1246.83926346, 1300.70152578, 1354.5637881 , 1408.42605042]), 'area': array([500.0, 565.2173913043479, 630.4347826086956, 695.6521739130435,
       760.8695652173913, 826.0869565217391, 891.304347826087,
       9

In [ ]:
# user input parameters

# boolean parameter to dictate whether code is run
run = False

# boolean parameter to dictate use of print statements
debug = False

# boolean parameter to dictate whether results are written to .csv files
write = False

In [ ]:
# function declaration

# input: timestep (timestep), reference name of a feature of interest (feature_ref), and dataframe dictionary to reference (well_dict)
# output: dictionary mapping a well's id number to its list of values of a given feature at an individual timestep
def get_timestep_population_dict(timestep, feature_ref, well_dict):
    population_dict = {key:[] for key in timestep_list}
    for well in well_dict:
        well_df = well_dict[well]
        timestep_vals = list((well_df.loc[well_df[timestep_ref] == timestep])[feature_ref])
        population_dict[well] = timestep_vals
    return population_dict

# input: reference name of a feature of interest (feature_ref), set of bin edges (bins), dataframe dictionary to reference (well_dict), and an optional path to a .csv file (csv_fpath)
# output: list of mutual information values calcualted from the given feature and wells      
def get_mi_list(feature_ref, bins, well_dict, csv_fpath=None):
    mi_list = []
    for timestep in timestep_list:
        population_dict = get_timestep_population_dict(timestep, feature_ref, well_dict)
        subset_size = int(ncells_avg_dict[timestep]//len(well_dict))
        mi = calculate_mi(population_dict, subset_size, bins)
        mi_list.append(mi)
        if csv_fpath != None:
            append_row_csv(csv_fpath, [mi])
    return mi_list

In [ ]:
# `main()` function declaration

# intended use in coordination with `pairwise_mi_plot.ipynb`
# requires: import of `data_processing.ipynb`, functions from `mi_calculation.ipynb`, and user input parameters specified above
# output: None (assigns new global variable `pairwise_dict` and writes to specified output .csv files)

def main():
    
    # dictionary mapping each feature to its optimal bin edge configuration
    global bin_edges_dict
    bin_edges_dict = create_bin_edges_dict()
    
    # create all pairs of wells
    pairs = [(w1, w2) for i, w1 in enumerate(well_id_list) for w2 in well_id_list[i + 1:]]
    
    # dictionary mapping each well to a dictionary of each of its pairwise mutual information values
    global pairwise_dict
    pairwise_dict = {key:{} for key in pairs}
    
    for (w1, w2) in pairs:
        
        # create a dictionary of well datafeames
        well_dict = {}
        well_dict[w1] = experiment_dict[w1]
        well_dict[w2] = experiment_dict[w2]
        
        for (name, ref) in zip(feature_name_list, feature_ref_list):
            
            # get a list of mutual information values for a given feature
            if write:
                csv_fpath = results_fpath + pairwise_mi_ext + f'{name}_{w1}_{w1}_mi_list.csv'
                append_row_csv(csv_fpath, ['mi'])
                mi_list = get_mi_list(ref, bin_edges_dict[name], well_dict, csv_fpath)
                
            else:
                mi_list = get_mi_list(ref, bin_edges_dict[name], well_dict)
            
            pairwise_dict[pair][name] = mi_list
    
    if debug:
        print('pairwise_dict:', type(pairwise_dict), '\n', pairwise_dict)
        
    return

In [ ]:
# call to `main()` function

if run:
    main()

In [10]:
print(pairwise_dict)

{('01', '03'): {'gfp': [0.15113319220343868, 0.1317685141598166, 0.180043956925791, 0.1498286291966595, 0.15677130516324045, 0.14214790358637508, 0.17497507888069586, 0.17950128588654812, 0.19246282739766962, 0.19142442014444883, 0.1638780475873889, 0.13698089543805136, 0.09022680015032436, 0.1468205359134711, 0.14740174068268752, 0.15469727737289407, 0.1639115973523606, 0.15061200294887533, 0.1190061312430235, 0.10151718984141644, 0.11904656051584928, 0.07154555890148884, 0.10476281365277229, 0.0428414276455511, 0.08134431780393991, 0.04238201017996768, 0.0028857832780988524, -0.010730836657316, 0.051610874567373344, 0.049036150502753006, 0.024526368427491674, 0.017480961887881286, 0.04692122690399003, 0.04769365457536945, 0.05992355659171511, 0.08568366803443173, 0.049577406328487506, 0.07235717740511216, 0.08694907246851402, 0.1319381217926252, 0.09001520669003393, 0.09337500705967794, 0.10943204100966475, 0.057154824877761135, 0.08423246991260158, 0.058583293666132574, 0.0538464160